En este módulo principal, se procede a integrar las funcionalidades definidas en cada uno de los endpoints individuales. A través de la utilización de app.include_router(), se incorporan los enrutadores correspondientes a la aplicación FastAPI, estableciendo de este modo un único punto de acceso para todas las operaciones expuestas por la API. Esta estructura modular facilita la gestión y el mantenimiento de la aplicación, permitiendo una escalabilidad eficiente al agregar nuevos endpoints en el futuro.


In [71]:
from fastapi import FastAPI

app = FastAPI()


In [72]:
from fastapi import FastAPI, HTTPException
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


#### Consulta 1 : ####
Esta consulta retorna un conjunto de datos estructurado en forma de tabla. Esta tabla proporciona un desglose cuantitativo del contenido ofrecido gratuitamente por cada empresa desarrolladora, especificando el número total de ítems y su correspondiente porcentaje respecto al total de ítems ofrecidos por dicha empresa en un determinado año.


In [73]:
import pandas as pd
import csv


Procedemos a importar el archivo con el cual vamos a trabajar.

In [74]:
df = pd.read_parquet("output_steam_games_limpio.parquet")

se generó una réplica del archivo

In [75]:
df_games_copy = df.copy()

Se implementa una función personalizada con el propósito de ejecutar la consulta deseada.

In [82]:
import pandas as pd
import json

def developer(desarrollador: str, df: pd.DataFrame):
    
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce') # convertimos "release_date" a fecha

    developer_df = df[df['developer'] == desarrollador]

    # Agrupamos por año y contamos ls elementos en cada grupo
    items_por_año = developer_df.groupby(df['release_date'].dt.year).size().reset_index(name='Cantidad de Items')

    # Calcular el porcentaje de contenido gratuito para cada año
    gratis_por_año = developer_df[developer_df['price'] == 0].groupby(df['release_date'].dt.year).size().reset_index(name='Contenido Free')

    # Combinar los resultados en un DataFrame final
    resultado_df = items_por_año.merge(gratis_por_año, on='release_date', how='left')
    #resultado_df['Contenido Free'] = (resultado_df['Contenido Free'] / resultado_df['Cantidad de Items'] * 100).fillna(0).astype(int).astype(str) + '%'

    # Renombrar la columna de años
    resultado_df.rename(columns={'release_date': 'Año'}, inplace=True)

    return resultado_df

In [83]:
desarrollador = input("cargue el desarrollador: ")
consulta = developer(desarrollador, df_games_copy)
print(consulta)

Empty DataFrame
Columns: [Año, Cantidad de Items, Contenido Free]
Index: []


#### Consulta 2 : ####
Esta consulta genera un diccionario que condensa la información relevante de cada usuario, incluyendo el monto total invertido, el porcentaje de recomendaciones basado en la columna 'reviews.recommend' y el número de artículos adquiridos.

In [42]:
df_games = pd.read_parquet("output_steam_games_limpio.parquet")
df_items = pd.read_parquet("australian_users_items_limpio.parquet")
df_reviews = pd.read_parquet("australian_user_reviews_limpio.parquet")

se procedió a elaborar una copia de trabajo.

In [11]:
games_copy = df_games.copy()
items_copy = df_items.copy()
reviews_copy = df_reviews.copy()

Se ha diseñado una función específica para llevar a cabo la consulta requerida, la cual permite obtener la información precisa necesaria para el análisis.

In [44]:
def userdata(User_id, items_copy, games_copy, reviews_copy):
    
    items_copy['user_id'] = items_copy['user_id'].astype(str) #convertimos "uder_id" a str
    reviews_copy['user_id'] = reviews_copy['user_id'].astype(str)

    user_items = items_copy[items_copy['user_id'] == str(User_id)]  #filtramos df_items

    games_copy['price'] = pd.to_numeric(games_copy['price'], errors='coerce') #calculamos el dinero que gasto el usuario y convertimos "price" a numerico
    dinero = user_items.merge(games_copy[['id', 'price']], left_on='item_id', right_on='id')['price'].sum()

    total_items = user_items['items_count'].sum() #cant de items del usuario

    user_reviews = reviews_copy[reviews_copy['user_id'] == str(User_id)] #filtramos df_reviews

    if user_reviews.shape[0] > 0: # calculo en prcentajede recomendacion promedio del usuario
        porcentaje_recomendacion = (user_reviews['recommend'].sum() / user_reviews.shape[0]) * 100
    else:
        porcentaje_recomendacion = 0

#diccionario de resultados
    result = {
        "Usuario": str(User_id),
        "Dinero gastado": f"{dinero:.2f} USD", # formato a2 decimales
        "% de recomendación": f"{porcentaje_recomendacion:.2f}%",
        "Cantidad de items": total_items
    }

    return result

In [45]:
usuario = input("Ingrese id de usuario: ")
resultado = userdata(usuario, items_copy, games_copy, reviews_copy)
print(resultado)

{'Usuario': '545', 'Dinero gastado': '0.00 USD', '% de recomendación': '0.00%', 'Cantidad de items': 0}


#### Consulta 3 : ####

Esta consulta permite identificar al usuario que ha dedicado más horas de juego a un género específico, así como obtener un listado de las horas acumuladas por año de lanzamiento de los juegos.

In [46]:
df_games = pd.read_parquet("output_steam_games_limpio.parquet")
df_items = pd.read_parquet("australian_users_items_limpio.parquet")

 se procedió a elaborar una copia de trabajo.

In [47]:
df_games_copy = df_games.copy()
df_items_copy = df_items.copy()

Se ha diseñado una función específica para llevar a cabo la consulta requerida, la cual permite obtener la información precisa necesaria para el análisis.

In [48]:
def UserForGenre(genero: str, df_games_copy, df_items_copy):
    # Verificar si la columna 'genre' está presente en el DataFrame df_games_copy
    if 'genres' not in df_games_copy.columns:
        raise ValueError("El DataFrame df_games_copy no tiene una columna llamada 'genre'.")

    # Convertir la columna 'release_date' a tipo datetime y extraer el año
    df_games_copy['Año'] = pd.to_datetime(df_games_copy['release_date'], errors='coerce').dt.year

    # Filtrar df_games_copy por el género dado
    juegos_genero = df_games_copy[df_games_copy['genres'] == genero]

    # Unir el DataFrame filtrado con df_user_items
    juegos_usuario = juegos_genero.merge(df_items_copy, left_on='id', right_on='item_id')

    # Calcular las horas jugadas por usuario para cada juego
    horas_por_usuario = juegos_usuario.groupby('user_id')['playtime_forever'].sum().reset_index()

    # Verificar si hay datos para el género dado
    if horas_por_usuario.empty:
        return {"Mensaje": f"No hay datos para el género '{genero}'."}

    # Encontrar el usuario con más horas jugadas
    usuario_max_horas = horas_por_usuario.loc[horas_por_usuario['playtime_forever'].idxmax()]['user_id']

    # Calcular la acumulación de horas jugadas por año de lanzamiento para el género dado
    horas_por_año = juegos_usuario.groupby('Año')['playtime_forever'].sum().reset_index()
    horas_por_año.rename(columns={'playtime_forever': 'Horas'}, inplace=True)
    horas_por_año['Año'] = horas_por_año['Año'].astype(int)  # Convertir el año a número entero
    horas_por_año = horas_por_año.to_dict('records')

    # Crear el diccionario de resultados
    result = {
        "Usuario con más horas jugadas para {}: ".format(genero): usuario_max_horas,
        "Horas jugadas": horas_por_año
    }

    return result

In [49]:
resultado = UserForGenre("Simulation", df_games_copy, df_items_copy)
print(resultado)

{'Usuario con más horas jugadas para Simulation: ': '76561198038513068', 'Horas jugadas': [{'Año': 1987, 'Horas': 0.0}, {'Año': 1988, 'Horas': 1.0}, {'Año': 1989, 'Horas': 0.0}, {'Año': 1990, 'Horas': 878.0}, {'Año': 1991, 'Horas': 144.0}, {'Año': 1992, 'Horas': 11.0}, {'Año': 1994, 'Horas': 4127.0}, {'Año': 1995, 'Horas': 980.0}, {'Año': 1998, 'Horas': 4361.0}, {'Año': 1999, 'Horas': 130537.0}, {'Año': 2000, 'Horas': 12342.0}, {'Año': 2001, 'Horas': 215993.0}, {'Año': 2002, 'Horas': 294785.0}, {'Año': 2003, 'Horas': 1363396.0}, {'Año': 2004, 'Horas': 101930.0}, {'Año': 2005, 'Horas': 80704.0}, {'Año': 2006, 'Horas': 2691671.0}, {'Año': 2007, 'Horas': 172230.0}, {'Año': 2008, 'Horas': 3798669.0}, {'Año': 2009, 'Horas': 13880055.0}, {'Año': 2010, 'Horas': 1124125.0}, {'Año': 2011, 'Horas': 3371092.0}, {'Año': 2012, 'Horas': 369703.0}, {'Año': 2013, 'Horas': 16534471.0}, {'Año': 2014, 'Horas': 3738916.0}, {'Año': 2015, 'Horas': 9669002.0}, {'Año': 2016, 'Horas': 5873778.0}, {'Año': 2017,

#### Consulta 4 : ####
El objetivo de esta consulta es identificar a los tres estudios de desarrollo de videojuegos más valorados por la comunidad de jugadores en un año específico. La métrica empleada para la selección es la cantidad de recomendaciones positivas recibidas por sus juegos.

In [50]:
def UsersBestDeveloper(año: int):

    df_games = pd.read_parquet("output_steam_games_limpio.parquet")
    df_reviews = pd.read_parquet("australian_user_reviews_limpio.parquet")
    df_sentimientos = pd.read_parquet ("user_reviews_analisis_ de_sentimientos.parquet")

    df_games_copy = df_games.copy()
    df_reviews_copy = df_reviews.copy()
    df_sentimientos_copy = df_sentimientos.copy()

    # Verificar si el año es igual a -1 y mostrar un mensaje personalizado
    if año == -1:
        return "El año ingresado es -1, lo cual no es válido."

In [51]:
df_games = pd.read_parquet("output_steam_games_limpio.parquet")
df_reviews = pd.read_parquet("australian_user_reviews_limpio.parquet")
df_sentimientos = pd.read_parquet("user_reviews_analisis_ de_sentimientos.parquet")

se procedió a elaborar una copia de trabajo.

In [52]:
df_games_copy = df_games.copy()
df_reviews_copy = df_reviews.copy()
df_sentimientos_copy = df_sentimientos.copy()

se procedió a la concatenación de los dos conjuntos de datos: el referente al análisis de sentimiento y el vinculado a los juegos. De esta manera, se integraron en una única tabla las variables de interés para la investigación.

In [56]:
df_fusionar = df_sentimientos_copy.merge(df_games_copy[['id', 'developer']], left_on='item_id', right_on='id', how='left')
df_fusionar

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analysis,id,developer
0,76561197970982479,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,2,2,1250,Tripwire Interactive
1,76561197970982479,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,2,2,22200,ACE Team
2,76561197970982479,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,2,2,NaN,NaN
3,js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,2,2,NaN,NaN
4,js41637,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,2,2,227300,SCS Software
...,...,...,...,...,...,...,...,...,...,...,...
59356,76561198312638244,None,Posted July 10.,None,70,No ratings yet,True,2,2,70,Valve
59357,76561198312638244,None,Posted July 8.,None,362890,No ratings yet,True,2,2,362890,Crowbar Collective
59358,LydiaMorley,1 person found this review funny,Posted July 3.,None,273110,1 of 2 people (50%) found this review helpful,True,2,2,273110,Nexon
59359,LydiaMorley,None,Posted July 20.,None,730,No ratings yet,True,1,1,730,Valve


 se procedió a la creación de una nueva columna denominada 'year_posted'. Esta columna contiene exclusivamente el año extraído del campo original 'posted', el cual sera posteriormente eliminado.

In [57]:
# Define un patrón regex para extraer el año (cuatro dígitos)
pattern = r'(\b\d{4}\b)'

# Extrae el año de cada celda en la columna 'posted' usando regex
df_fusionar['year_posted'] = df_fusionar['posted'].str.extract(pattern)

# Elimina la columna original 'posted'
df_fusionar.drop(columns=['posted'], inplace=True)

A continuacion reordenamos las columnas

In [58]:
nuevo_orden_columnas = [
    'user_id',
    'item_id',
    'recommend',
    'sentiment_analysis',
    'developer',
    'year_posted'
]

# Reordena las columnas del DataFrame con el nuevo orden
df_merged = df_fusionar[nuevo_orden_columnas]

df_merged

,user_id,item_id,recommend,sentiment_analysis,developer,year_posted
0,76561197970982479,1250,True,2,Tripwire Interactive,2011
1,76561197970982479,22200,True,2,ACE Team,2011
2,76561197970982479,43110,True,2,NaN,2011
3,js41637,251610,True,2,NaN,2014
4,js41637,227300,True,2,SCS Software,2013
...,...,...,...,...,...,...
59356,76561198312638244,70,True,2,Valve,NaN
59357,76561198312638244,362890,True,2,Crowbar Collective,NaN
59358,LydiaMorley,273110,True,2,Nexon,NaN
59359,LydiaMorley,730,True,1,Valve,NaN


Procedemos a delimitar la función que la consulta estaría destinada a cumplir.


In [59]:
def best_developer_year(df, año):
    df_año = df[df['year_posted'] == año] #filtramos de a cuerdo al año

    df_filtrado = df_año[df_año['recommend'] & (df_año['sentiment_analysis'] == df_año['sentiment_analysis'].max())] #filtramos por recomendaciones verdaderas y analisis de sentimiento mas alto

    top_developers = df_filtrado.groupby('developer')['recommend'].sum() #agrupo por desarrllador y cuento recomendaciones

    top_developers = top_developers.sort_values(ascending=False) #ordeno desarrolladores pornum de recomendacion (DESC)

    top_3_developers = top_developers.head(3)#top 3

    # creo dicc con top 3
    top_developers_dict = {}
    for i, (developer, recomendaciones) in enumerate(top_3_developers.items(), 1):
        puesto = f"Puesto {i}"
        top_developers_dict[puesto] = developer

    return top_developers_dict

In [60]:
año = 2015
top_3_desarrolladores = best_developer_year(df_merged, año)

# Imprimir el top 3 de desarrolladores para el año dado
print(f"Top 3 de desarrolladores para el año {año}: {top_3_desarrolladores}")

Top 3 de desarrolladores para el año 2015: {}


Se procedió a exportar el conjunto de datos 'df_merged' al formato Parquet con el fin de optimizar su integración con la API.

In [61]:
df_merged.to_parquet("reviews_merged.parquet", index=False)

#### Consulta 5 : ####
La consulta genera un diccionario en el cual las claves corresponden a los nombres de los desarrolladores. El valor asociado a cada clave es una lista que contiene dos elementos: el número total de reseñas positivas y el número total de reseñas negativas. En aquellos casos donde no se dispone de información sobre un desarrollador en particular o si las reseñas no han sido categorizadas, se retorna el mensaje 'No se encontró información sobre el desarrollador. 

In [62]:
games = pd.read_parquet("output_steam_games_limpio.parquet")
sentiment = pd.read_parquet("user_reviews_analisis_ de_sentimientos.parquet")

Realizamos una copia de los mismos para evitar modificaciones  accidentales

In [63]:
games_copy = games.copy()
sentiment_copy = sentiment.copy()

Procedemos a delimitar la función que la consulta estaría destinada a cumplir.


In [64]:
def developer_reviews_analysis(desarrolladora):
    merged_data = pd.merge(sentiment_copy, games_copy, left_on='item_id', right_on='id')#combino conj de datos en las clumnas apropiadas

#Filtro filas donde el puntaje de sentimiento es positivo (2) o negativo (0)
    filtered_data = merged_data[merged_data['sentiment_analysis'] != 1]  # Excluir sentimiento neutral

#Agrupamos por desarrolladora y puntaje de sentimiento
    grouped_data = filtered_data.groupby(['developer', 'sentiment_analysis']).size().unstack(fill_value=0)

    if desarrolladora in grouped_data.index:# verifico si la desarrolladora esta presente
        # Extraemos cantidad de reseñas positivas y negativas
        developer_reviews = grouped_data.loc[desarrolladora]

        # Convertimos  a formato de lista con claves especificadas
        developer_reviews_list = [
            {"Negativas": developer_reviews.get(0, 0)},
            {"Positivas": developer_reviews.get(2, 0)}
        ]

        return {desarrolladora: developer_reviews_list}
    else:
        return f"No se encontró información sobre la desarrolladora {desarrolladora}"

In [65]:
developer = input("Ingrese nombre del desarrolladora: ")
resultado = developer_reviews_analysis(developer)
print(resultado)

No se encontró información sobre la desarrolladora 56
